In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

from spacy.lang.pl import Polish

import string 
import re

import seaborn as sns
from collections import  Counter
from matplotlib.pyplot import figure

In [ ]:
nlp = Polish()

### Data Loading

In [ ]:
!wget "http://2019.poleval.pl/task6/task_6-2.zip"

--2022-05-07 13:34:39--  http://2019.poleval.pl/task6/task_6-2.zip
Resolving 2019.poleval.pl (2019.poleval.pl)... 213.135.36.94
Connecting to 2019.poleval.pl (2019.poleval.pl)|213.135.36.94|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 340096 (332K) [application/zip]
Saving to: ‘task_6-2.zip’

task_6-2.zip        100%[===================>] 332.12K  --.-KB/s    in 0.1s    

2022-05-07 13:34:39 (3.08 MB/s) - ‘task_6-2.zip’ saved [340096/340096]



In [ ]:
!unzip "/content/task_6-2.zip" -d "/content/data"

Archive:  /content/task_6-2.zip
  inflating: /content/data/training_set_clean_only_text.txt  
  inflating: /content/data/training_set_clean_only_tags.txt  


In [ ]:
original_tags = pd.read_csv('/content/data/training_set_clean_only_tags.txt', header = None)
with open('/content/data/training_set_clean_only_text.txt', 'r') as textFile:
  lines = textFile.readlines()
original_text = pd.DataFrame(lines)

df = pd.concat([original_text,original_tags], axis=1)
df.columns =['Text', 'Tag']

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10041 entries, 0 to 10040
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   Text    10041 non-null  object
 1   Tag     10041 non-null  int64 
dtypes: int64(1), object(1)
memory usage: 157.0+ KB


# Data Preprocessing

In [ ]:
RE_EMOJI = re.compile('[\U00010000-\U0010ffff]', flags=re.UNICODE)

def preprocess_text(text):
  # remove whitespaces
  text = ' '.join(text.split())

  # tokenize
  doc = [tok for tok in nlp(text)]

  # remove RT @...
  if str(doc[0]) == "RT": 
    doc.pop(0)

  while str(doc[0]) == "@anonymized_account":
    doc.pop(0)
  while str(doc[-1]) == "@anonymized_account":
    doc.pop()

  # remove punctuation
  doc = [t for t in doc if t.text not in string.punctuation]
  
  # doc = [tok for tok in doc if not tok.is_stop]

  doc = [tok.lower_ for tok in doc]
  # doc = [RE_EMOJI.sub(r'', str_text) for str_text in doc]
  doc = [tok for tok in doc if len(tok) > 0]

  return doc

In [ ]:
df['Tokens'] = df['Text'].map(lambda x: preprocess_text(x))

In [ ]:
df[df["Tag"] == 1].head()

,Text,Tag,Tokens
44,@anonymized_account @anonymized_account @anony...,1,"[jak, narazie, to, masz, przywidzenia, co, nie..."
53,Nikt nigdy nie rozsiewał takiego smrodu jak @a...,1,"[nikt, nigdy, nie, rozsiewał, takiego, smrodu,..."
153,@anonymized_account Żal ci biedaku??? Gdyby ni...,1,"[żal, ci, biedaku, gdyby, nie, kaczyński, to, ..."
154,RT @anonymized_account @anonymized_account Żal...,1,"[żal, ci, biedaku, gdyby, nie, kaczyński, to, ..."
161,"@anonymized_account Jakiej pracy, ty już pajac...",1,"[jakiej, pracy, ty, już, pajacu, nie, wiesz, n..."


## Tfidf

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
s= ['👋 Welcome', '👋 Welcome']

def identity_tokenizer(text):
    return [toc for toc in text if bool(RE_EMOJI.match(toc))]

v = TfidfVectorizer(tokenizer=identity_tokenizer, lowercase=False) # token_pattern=r'[^\s]+'
tf_result = v.fit_transform(df['Tokens'])

tf_emojis_tensor = torch.from_numpy(tf_result.todense())

In [ ]:
v.get_feature_names_out()

array(['🆘', '🇦', '🇧', '🇬', '🇱', '🇵', '🌞', '🌧', '🌴', '🌹', '🌼', '🍀', '🍻',
       '🍼', '🍾', '🎁', '🎂', '🎈', '🎉', '🎤', '🎶', '🎷', '🎹', '🎺', '🏆', '🏴',
       '🏻', '🏻\u200d♀️', '🏻✌🏻', '🏻👍🏻',
       '🏼\\n\\n#ustawadegradacyjna\\n\\nhttps://t.co/a5lffolmwj',
       '🏼\u200d♀️', '🐇', '🐑', '🐕', '🐙', '🐝', '🐰', '🐱', '🐷', '🐻', '👇', '👊',
       '👋', '👌', '👍', '👎', '👏', '👐', '👪', '👰', '💁', '💐', '💓', '💔', '💕',
       '💛', '💝', '💞', '💪', '📚', '📺', '🔊', '🔜', '🔝', '🔥', '🔧', '🔪', '🔫',
       '🔯', '🕵', '🖒', '😀', '😁', '😁\\n', '😁😁😁😁😁.no', '😂',
       '😂😂😂\\n#woronicza17', '😃', '😄', '😅', '😆', '😇', '😉', '😊', '😋', '😍',
       '😎', '😐', '😑', '😒', '😓', '😔', '😖', '😘', '😛', '😜', '😝', '😞', '😡',
       '😢', '😣', '😥', '😧', '😨', '😩', '😬', '😭', '😮', '😯', '😱', '😳', '😷',
       '🙂', '🙃', '🙄', '🙆', '🙇', '🙈', '🙉', '🙊', '🙋', '🙌', '🙏', '🚜', '🤐',
       '🤔', '🤗', '🤣', '🤦', '🤨', '🤪', '🤭', '🤷', '🥀', '🥂', '🦀', '🧐',
       '\U000e0067\U000e0062\U000e0065\U000e006e\U000e0067\U000e007f'],
      dtype=object)

## TFIDF



### TFIDF -> RF

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split

data_frame = df
X1_train, X1_test, Y1_train, Y1_test = train_test_split(data_frame['Text'], data_frame['Tag'], test_size = 0.3, random_state=0, shuffle = True, stratify=data_frame['Tag'])
vectorizer = TfidfVectorizer()
X1_train_vect = vectorizer.fit_transform(X1_train)
Y1 = df.loc[:,'Tag'].values

In [ ]:
from sklearn.pipeline import Pipeline
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.ensemble import RandomForestClassifier

rf = Pipeline([('tfidf', TfidfVectorizer(token_pattern=r'[^\s]+')), ('rf', RandomForestClassifier())])
rf.fit(X1_train, Y1_train)
y_pred = rf.predict(X1_test)
print(pd.crosstab(Y1_test,y_pred,rownames=['Actual'],colnames=['Predicted']))
print(classification_report(Y1_test, y_pred))

Predicted     0  1   2
Actual                
0          2758  0   0
1            70  4   2
2           146  0  33
              precision    recall  f1-score   support

           0       0.93      1.00      0.96      2758
           1       1.00      0.05      0.10        76
           2       0.94      0.18      0.31       179

    accuracy                           0.93      3013
   macro avg       0.96      0.41      0.46      3013
weighted avg       0.93      0.93      0.90      3013



### TFIDF Emoji Info

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
s= ['👋 Welcome', '👋 Welcome']

def identity_tokenizer(text):
    return [toc for toc in text if bool(RE_EMOJI.match(toc))]

v = TfidfVectorizer(tokenizer=identity_tokenizer, lowercase=False) # token_pattern=r'[^\s]+'
tf_result = v.fit_transform(df['Tokens'])

tf_emojis_tensor = torch.from_numpy(tf_result.todense())

In [ ]:
v.get_feature_names_out()

array(['🆘', '🇦', '🇧', '🇬', '🇱', '🇵', '🌞', '🌧', '🌴', '🌹', '🌼', '🍀', '🍻',
       '🍼', '🍾', '🎁', '🎂', '🎈', '🎉', '🎤', '🎶', '🎷', '🎹', '🎺', '🏆', '🏴',
       '🏻', '🏻\u200d♀️', '🏻✌🏻', '🏻👍🏻',
       '🏼\\n\\n#ustawadegradacyjna\\n\\nhttps://t.co/a5lffolmwj',
       '🏼\u200d♀️', '🐇', '🐑', '🐕', '🐙', '🐝', '🐰', '🐱', '🐷', '🐻', '👇', '👊',
       '👋', '👌', '👍', '👎', '👏', '👐', '👪', '👰', '💁', '💐', '💓', '💔', '💕',
       '💛', '💝', '💞', '💪', '📚', '📺', '🔊', '🔜', '🔝', '🔥', '🔧', '🔪', '🔫',
       '🔯', '🕵', '🖒', '😀', '😁', '😁\\n', '😁😁😁😁😁.no', '😂',
       '😂😂😂\\n#woronicza17', '😃', '😄', '😅', '😆', '😇', '😉', '😊', '😋', '😍',
       '😎', '😐', '😑', '😒', '😓', '😔', '😖', '😘', '😛', '😜', '😝', '😞', '😡',
       '😢', '😣', '😥', '😧', '😨', '😩', '😬', '😭', '😮', '😯', '😱', '😳', '😷',
       '🙂', '🙃', '🙄', '🙆', '🙇', '🙈', '🙉', '🙊', '🙋', '🙌', '🙏', '🚜', '🤐',
       '🤔', '🤗', '🤣', '🤦', '🤨', '🤪', '🤭', '🤷', '🥀', '🥂', '🦀', '🧐',
       '\U000e0067\U000e0062\U000e0065\U000e006e\U000e0067\U000e007f'],
      dtype=object)

# Model

## Installations

In [ ]:
!pip install transformers

In [ ]:
!pip install pytorch-lightning

     |████████████████████████████████| 584 kB 12.1 MB/s 
     |████████████████████████████████| 136 kB 39.6 MB/s 
     |████████████████████████████████| 409 kB 37.7 MB/s 
     |████████████████████████████████| 1.1 MB 44.7 MB/s 
     |████████████████████████████████| 94 kB 3.2 MB/s 
     |████████████████████████████████| 144 kB 51.6 MB/s 
     |████████████████████████████████| 271 kB 9.0 MB/s 


## Transformers

In [ ]:
from transformers import BertTokenizer, pipeline, BertModel

import torch

In [ ]:
tokenizer = BertTokenizer.from_pretrained("dkleczek/bert-base-polish-uncased-v1")
model_bert = BertModel.from_pretrained("dkleczek/bert-base-polish-uncased-v1")

Downloading:   0%|          | 0.00/483k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/112 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/2.00 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/478 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/507M [00:00<?, ?B/s]

Some weights of the model checkpoint at dkleczek/bert-base-polish-uncased-v1 were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.decoder.weight', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [ ]:
model_bert.eval();

## Dataset wrapper

In [ ]:
from torch.utils.data import Dataset


In [ ]:
class HateSpeechDataset(Dataset):
    def __init__(self, data_df, bert_tokenizer, tf_idf_emoji_matrix):
        self.data_df = data_df
        self.bert_tokenizer = bert_tokenizer
        self.tf_idf_emoji_matrix = tf_idf_emoji_matrix

    def __len__(self):
        return len(self.data_df)

    def __getitem__(self, idx):
        tf_emotes = torch.from_numpy(self.tf_idf_emoji_matrix[idx].todense().astype(np.float32)).flatten()
        label = self.data_df['Tag'][idx]
        tokenizer_output = self.bert_tokenizer([self.data_df['Text'][idx]], return_tensors = "pt", padding='max_length', max_length = 100, truncation = True) #is_split_into_words=True
        return tokenizer_output['input_ids'].flatten(), tokenizer_output['attention_mask'].flatten(), tf_emotes, torch.tensor(label).long() #907

In [ ]:
torch.manual_seed(42)

hs_dataset = HateSpeechDataset(df, tokenizer, tf_result)
train_size = int(len(hs_dataset)*0.8)
val_size = int(len(hs_dataset)*0.1)
train_ds, val_ds, test_ds = torch.utils.data.random_split(hs_dataset, 
                                                  [train_size, val_size, len(hs_dataset) - train_size - val_size])

In [ ]:
from torch.utils.data import DataLoader

train_loader = DataLoader(train_ds, batch_size=32, num_workers=2)

val_loader = DataLoader(val_ds, batch_size=32, num_workers=2)

test_loader = DataLoader(test_ds, batch_size=32, num_workers=2)

# Model: TFIdf + Bert -> MLP

In [ ]:
from torch import nn
import torch.nn.functional as F

import pytorch_lightning as pl

from pytorch_lightning.callbacks.early_stopping import EarlyStopping
from pytorch_lightning.callbacks import ModelCheckpoint

import torchmetrics

In [ ]:
class MLP(nn.Module):
    def __init__(self, dims, dropout = 0.5, activation = None):
        super().__init__()
        # assert len(dims) > 2, 'must have at least 3 dimensions, for dimension in and dimension out'
        activation = activation if activation is not None else nn.ReLU

        layers = []
        pairs = list(zip(dims[:-1], dims[1:]))

        for ind, (dim_in, dim_out) in enumerate(pairs):
            is_last = ind >= (len(pairs) - 1)
            layers.append(nn.Linear(dim_in, dim_out))
            if not is_last:
                layers.append(nn.Dropout(dropout))
                layers.append(activation())

        # layers.append(nn.Softmax())

        self.net = nn.Sequential(*layers)

    def forward(self, x):
        return self.net(x).flatten(1)

In [ ]:
class HateSpeechClassifier(nn.Module):
    def __init__(self, bert_model, classification_model):
        super().__init__()
        self.bert = bert_model
        self.classification_model = classification_model

    def forward(self, bert_tokens_ids, bert_tokens_attention, tfidf_features):
      with torch.no_grad():
        _, cls_embeds = self.bert(input_ids = bert_tokens_ids, attention_mask = bert_tokens_attention, return_dict=False)
      features_connected = torch.cat((cls_embeds, tfidf_features), 1)

      return self.classification_model(features_connected)

In [ ]:
df.Tag.value_counts()

0    9190
2     598
1     253
Name: Tag, dtype: int64

In [ ]:
class ModelLightining(pl.LightningModule):
    def __init__(self, model):
        super(ModelLightining, self).__init__()
        self.model = model

        metrics = torchmetrics.MetricCollection({
            "accuracy": torchmetrics.Accuracy(num_classes = 3), 
            "microF1": torchmetrics.F1Score(num_classes = 3, average = 'micro'), 
            "macroF1": torchmetrics.F1Score(num_classes = 3, average = 'macro')})

        self.train_metrics = metrics.clone(prefix='train_')
        self.valid_metrics = metrics.clone(prefix='val_')

        self.f1_per_class = torchmetrics.F1Score(num_classes = 3,average = None)

        self.weights = [0.05, 0.975, 0.95] # ?

    def forward(self, bert_tokens, bert_attention, tfidf):
        return self.model(bert_tokens, bert_attention, tfidf)

    def training_step(self, batch, batch_index):
        bert_tokens, bert_attention, tfidf, labels = batch

        output = self.forward(bert_tokens, bert_attention, tfidf)

        loss = F.cross_entropy(output, labels, torch.tensor(self.weights, device=self.device))

        metric_outputs = self.train_metrics(output, labels)
        self.log_dict(metric_outputs, on_step=False, on_epoch=True, prog_bar=True, logger=True)
        self.log("train_loss", loss, on_step=False, on_epoch=True, prog_bar=True, logger=True)
        return loss

    def validation_step(self, batch, batch_idx):
        bert_tokens, bert_attention, tfidf, labels = batch

        y_hat = self.forward(bert_tokens, bert_attention, tfidf)

        # if((labels!=0).any()):
        #   print(torch.cat([torch.unsqueeze(labels, 1), y_hat], dim = 1))

        loss = F.cross_entropy(y_hat, labels, torch.tensor(self.weights, device=self.device))
        metric_outputs = self.valid_metrics(y_hat, labels)
        f1_pc = self.f1_per_class(y_hat, labels)
        self.log_dict(metric_outputs, on_step=False, on_epoch=True, prog_bar=True, logger=True)
        self.log("val_loss", loss, on_step=False, on_epoch=True)
        return f1_pc

    def test_step(self, batch, batch_idx):
        bert_tokens, bert_attention, tfidf, labels = batch
        y_hat = self.forward(bert_tokens, bert_attention, tfidf)
        loss = F.cross_entropy(y_hat, labels, torch.tensor(self.weights, device=self.device))
        self.log("test_loss", loss)
        
    def validation_epoch_end(self, validation_step_outputs):
        all_acc = torch.stack(validation_step_outputs, dim = 0)
        print(f'\n Val F1 per class: {torch.nanmean(all_acc, dim = 0)}')
            

    def configure_optimizers(self):
        return torch.optim.Adam(self.model.parameters(), lr=1e-3)

In [ ]:
model_classifier = MLP([907, 3])

model = HateSpeechClassifier(model_bert, model_classifier)

model_pl = ModelLightining(model)

In [ ]:
early_stop_callback = EarlyStopping(monitor="val_loss", patience=5, verbose = True)
model_checkpoint_callback = ModelCheckpoint(monitor="val_loss", dirpath = 'MLP')
trainer = pl.Trainer(max_epochs=3, devices=1, accelerator="auto", 
                     callbacks=[early_stop_callback, model_checkpoint_callback])

GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


In [ ]:
trainer.fit(model_pl, train_dataloaders=train_loader, val_dataloaders=val_loader)

/usr/local/lib/python3.7/dist-packages/pytorch_lightning/callbacks/model_checkpoint.py:611: UserWarning: Checkpoint directory /content/MLP exists and is not empty.
  rank_zero_warn(f"Checkpoint directory {dirpath} exists and is not empty.")
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name          | Type                 | Params
-------------------------------------------------------
0 | model         | HateSpeechClassifier | 132 M 
1 | train_metrics | MetricCollection     | 0     
2 | valid_metrics | MetricCollection     | 0     
3 | f1_per_class  | F1Score              | 0     
-------------------------------------------------------
132 M     Trainable params
0         Non-trainable params
132 M     Total params
528.496   Total estimated model params size (MB)


Sanity Checking: 0it [00:00, ?it/s]


 Val F1 per class: tensor([0.9508, 0.0000, 0.0000], device='cuda:0')


Training: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Metric val_loss improved. New best score: 0.378



 Val F1 per class: tensor([0.9612, 0.0000, 0.0345], device='cuda:0')


/usr/local/lib/python3.7/dist-packages/pytorch_lightning/trainer/trainer.py:724: UserWarning: Detected KeyboardInterrupt, attempting graceful shutdown...
  rank_zero_warn("Detected KeyboardInterrupt, attempting graceful shutdown...")


In [ ]:
trainer.validate(ckpt_path = 'best', dataloaders = test_loader)

Restoring states from the checkpoint path at /content/MLP/epoch=13-step=3514.ckpt
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
Loaded model weights from checkpoint at /content/MLP/epoch=13-step=3514.ckpt


Validation: 0it [00:00, ?it/s]


 Val F1 per class: tensor([0.9059, 0.0846, 0.3253], device='cuda:0')
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
     Validate metric           DataLoader 0
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
      val_accuracy          0.8208954930305481
        val_loss            0.8080368041992188
       val_macroF1          0.45084166526794434
       val_microF1          0.8208954930305481
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────


[{'val_accuracy': 0.8208954930305481,
  'val_loss': 0.8080368041992188,
  'val_macroF1': 0.45084166526794434,
  'val_microF1': 0.8208954930305481}]